In [1]:
from ipywidgets import interact
import numpy as np
import pandas as pd
import scipy.stats as st
import matplotlib.pylab as plt

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import LinearColorMapper, ColumnDataSource
import bokeh.palettes as bkpal

output_notebook()

Loading BokehJS ...

### A binomial example

Coin flip example: flip a coin ten times, how many times it will come up head?

In [2]:
n0, p0 = 10, .5
x0 = np.arange(n0)
y0 = st.binom.pmf(x0, n0, p0)

fig0 = figure(title="Binomial PMF", plot_height=300, plot_width=600)
ax0 = fig0.vbar(x0, .5, y0);

def update0(p0=.1):
    ax0.data_source.data['top'] = st.binom.pmf(x0, n0, p0)

    push_notebook()

In [3]:
show(fig0, notebook_handle=True)
interact(update0, p0=(0, 1, 0.01));

interactive(children=(FloatSlider(value=0.1, description='p0', max=1.0, step=0.01), Output()), _dom_classes=('widget-interact',))

If the number of head is observed, we can infer the Binomial rate parameter, by plugging it into the PMF:
$\pi(x \mid n, p) = \binom{n}{x} p^x (1-p)^{n-x}$ 

In [4]:
k = 7
xp = np.linspace(0., 1., 200)
ll = st.binom.pmf(k, n0, xp)

fig1 = figure(title="Likelihood of rate parameter p in Binomial", plot_height=300, plot_width=600)
ax1 = fig1.line(xp, ll);

def update2(k=5):
    ax1.data_source.data['y'] = st.binom.pmf(k, n0, xp)
    print('Observed k = %d'%(k))
    push_notebook()

In [5]:
show(fig1, notebook_handle=True)
interact(update2, k=(0, n0, 1));

interactive(children=(IntSlider(value=5, description='k', max=10), Output()), _dom_classes=('widget-interact',))

Binomial distribution has two parameters, n and p. Thus we can vary both and see what are the possible outputs.  

Again: the entery of Binomial distribution with $\text{Binomial}(n, p)$ is a _family_ of distribution, once the n and p is fixed, it becomes a distribution of the outcome k.

In [6]:
n, p = 10, .5
nmax = 25
x = np.arange(nmax)
y = st.binom.pmf(x, n, p)

fig2 = figure(title="Binomial PMF", plot_height=300, plot_width=600)
ax2 = fig2.vbar(x, .5, y);

def update3(n=10, p=.1):
    x0 = ax2.data_source.data['x']
    y = st.binom.pmf(x0, n, p)
    ax2.data_source.data['top'] = y

    push_notebook()

In [7]:
show(fig2, notebook_handle=True)
interact(update3, n=(0, nmax, 1), p=(0, 1, 0.01));

interactive(children=(IntSlider(value=10, description='n', max=25), FloatSlider(value=0.1, description='p', max=1.0, step=0.01), Output()), _dom_classes=('widget-interact',))

Thus, when you observed a certain output (say 7 heads), both no information of n (number of coin flip) and p (rate parameter), then we are trying to infer 2 variables:



In [8]:
k, n = 7, 10
pv = np.linspace(0., 1., 50)
nv = np.arange(50)
pv_, nv_ = np.meshgrid(pv, nv)
ll = st.binom.pmf(k, nv_, pv_)

df = pd.DataFrame(dict(likelihood=ll.flatten(), p=pv_.flatten(), n=nv_.flatten()))
colors = bkpal.viridis(100)
mapper = LinearColorMapper(palette=colors, low=df.likelihood.min(), high=df.likelihood.max())
source = ColumnDataSource(df)

TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"
fig4 = figure(
    title="Likelihood of rate parameter n and p in Binomial",
    plot_height=500,
    plot_width=500,
    tools=TOOLS, toolbar_location='below')

fig4.grid.grid_line_color = None
fig4.axis.axis_line_color = None
fig4.axis.major_tick_line_color = None
fig4.axis.major_label_text_font_size = "5pt"
fig4.axis.major_label_standoff = 0

ax4 = fig4.rect(
    'p',
    'n',
    width=pv[1] * .99,
    height=1,
    source=source,
    fill_color={'field': 'likelihood',
                'transform': mapper},
    line_color=None)

from bokeh.models import HoverTool
fig4.select_one(HoverTool).tooltips = [
    ('n', '@n'),
    ('p', '@p'),
    ('like', '@likelihood'),
]

def update4(k=7):
    ll = st.binom.pmf(k, nv_, pv_)
    ax4.data_source.data['likelihood'] = ll.flatten()

    push_notebook()

In [9]:
show(fig4, notebook_handle=True)
interact(update4, k=(0, 25, 1));

interactive(children=(IntSlider(value=7, description='k', max=25), Output()), _dom_classes=('widget-interact',))

$f(x, n, p) = \binom{n}{x} p^x (1-p)^{n-x}$  

### Another example: Gaussian with parameter $\mu$ and $\sigma$

In [10]:
obs = 0.
muv = np.linspace(-10., 10., 50)
sdv = np.linspace(0.001, 5., 50)
muv_, sdv_ = np.meshgrid(muv, sdv)
ll = st.norm.pdf(obs, muv_, sdv_)

df2 = pd.DataFrame(dict(likelihood=ll.flatten(), mu=muv_.flatten(), sd=sdv_.flatten()))
colors = bkpal.viridis(100)
lower_ll, upper_ll = np.percentile(ll.flatten(), [1, 99])
mapper = LinearColorMapper(palette=colors, low=lower_ll, high=upper_ll)
source2 = ColumnDataSource(df2)

In [11]:
TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"
fig5 = figure(
    title="Likelihood of mu and sigma in Gaussian",
    plot_height=500,
    plot_width=500,
    tools=TOOLS, toolbar_location='below')

fig5.grid.grid_line_color = None
fig5.axis.axis_line_color = None
fig5.axis.major_tick_line_color = None
fig5.axis.major_label_text_font_size = "5pt"
fig5.axis.major_label_standoff = 0

ax5 = fig5.rect(
    'mu',
    'sd',
    width=(muv[1]-muv[0]) * .99,
    height=(sdv[1]-sdv[0]) * .99,
    source=source2,
    fill_color={'field': 'likelihood',
                'transform': mapper},
    line_color=None)

fig5.select_one(HoverTool).tooltips = [
    ('mu', '@mu'),
    ('sd', '@sd'),
    ('like', '@likelihood'),
]

def update5(obs=0):
    ll = st.norm.pdf(obs, muv_, sdv_)
    ax5.data_source.data['likelihood'] = ll.flatten()

    push_notebook()

In [ ]:
show(fig5, notebook_handle=True)
interact(update5, obs=(-10, 10, .01));